In [575]:
%load_ext autoreload
%autoreload 2

In [457]:
import logging
import collections
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open 
import pandas as pd
import numpy as np
from numpy import random
from collections import Counter
import heapq #check n largest
import json
import warnings
from sqlalchemy import create_engine 
import spacy
import nltk
import seaborn as sns

In [458]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [459]:
bath_path = r''

computer_security_and_networks = json.load(open(bath_path + r'computer-security-and-networks.json', encoding='utf8'))
algorithms = json.load(open(bath_path + r'algorithms.json', encoding='utf8'))
cloud_computing = json.load(open(bath_path + r'cloud-computing.json', encoding='utf8'))
data_management = json.load(open(bath_path + r'data-management.json', encoding='utf8'))
design_and_product = json.load(open(bath_path + r'design-and-product.json', encoding='utf8'))
electrical_engineering = json.load(open(bath_path + r'electrical-engineering.json', encoding='utf8'))
math_and_logic = json.load(open(bath_path + r'math-and-logic.json', encoding='utf8'))
mobile_and_web_development = json.load(open(bath_path + r'mobile-and-web-development.json', encoding='utf8'))
networking = json.load(open(bath_path + r'networking.json', encoding='utf8'))
probability_and_statistics = json.load(open(bath_path + r'probability-and-statistics.json', encoding='utf8'))
security = json.load(open(bath_path + r'security.json', encoding='utf8'))
machine_learning = json.load(open(bath_path + r'machine-learning.json', encoding='utf8'))
data_analysis = json.load(open(bath_path + r'data-analysis.json', encoding='utf8'))
software_development = json.load(open(bath_path + r'software-development.json', encoding='utf8'))

all_data = computer_security_and_networks + \
algorithms + \
cloud_computing + \
data_management + \
design_and_product + \
electrical_engineering + \
math_and_logic + \
mobile_and_web_development + \
networking + \
probability_and_statistics + \
security + \
machine_learning + \
data_analysis + \
software_development

In [460]:
len(all_data)

881

In [461]:
title=[]; week_data=[]; target_audience=[]; created_by=[]; teach_by=[]
week_data_title=[]; course_info=[]; department=[]; week_data_desc=[]
package_num=[];package_name=[]; level=[]; rating=[]; index=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    for i in range(len(all_data[n]['teach_by'])):
            title.append(all_data[n]['title'])
            target_audience.append(all_data[n]['target_audience'])
            created_by.append(all_data[n]['created_by'])
            course_info.append(all_data[n]['course_info'])
            package_num.append(all_data[n]['package_num'])
            package_name.append(all_data[n]['package_name'])
            level.append(all_data[n]['level'])
            rating.append(all_data[n]['rating'])
            teach_by.append(list((all_data[n]['teach_by'][i].values()))[0])  
            department.append(list((all_data[n]['teach_by'][i].values()))[1])

In [462]:
data = pd.DataFrame({'title': title,
                    'target_audience': target_audience,
                    'created_by' : created_by,  
                    'course_info': course_info,
                    'teach_by': teach_by,
                     'department': department,
                     'level': level,
                     'rating': rating,
                     'package_num': package_num,
                     'package_name': package_name          
                    },
                    columns = ['title', 'course_info', 'target_audience', 'created_by','teach_by',
                               'department','package_num','package_name', 'level','rating'])

In [463]:
data.head()

,title,course_info,target_audience,created_by,teach_by,department,package_num,package_name,level,rating
0,Bitcoin and Cryptocurrency Technologies,About this course: To really understand what i...,None,Princeton University,Arvind Narayanan,Computer Science,None,None,None,4.7
1,Cryptography I,About this course: Cryptography is an indispen...,None,Stanford University,Dan Boneh,Computer Science,None,None,None,4.8
2,Google Cloud Platform Fundamentals: Core Infra...,About this course: <<< ATTENTION >>> Learners ...,Who is this class for: This specialization is ...,Google Cloud,Google Cloud Training,None,1,Architecting with Google Cloud Platform Specia...,Intermediate,4.6
3,G Suite Administrator Fundamentals,About this course: G Suite Administrator Funda...,Who is this class for: This class is intended ...,Google Cloud,Google Cloud Training,None,None,None,Beginner,4.5
4,Usable Security,About this course: This course focuses on how ...,None,"University of Maryland, College Park",Jennifer Golbeck,Human-Computer Interaction Lab,1,Cybersecurity Specialization,None,4.5


In [464]:
data =  data.drop_duplicates()

In [465]:
for i, row in data.iterrows():
    if not isEnglish(row.title):
        data.drop(i, inplace=True)

In [466]:
len(data.title.unique())

608

In [467]:
title=[]; created_by=[]; course_info=[]
data_title=[]; data_desc=[]; data_video=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    week_data_title=[];  week_data_desc=[];week_data_video=[]
    title.append(all_data[n]['title'])
    created_by.append(all_data[n]['created_by'])
    for i in range(len(all_data[n]['week_data'])):
        week_data_desc.append(all_data[n]['week_data'][i]['description'])
        week_data_title.append(all_data[n]['week_data'][i]['title'])
        week_data_video += all_data[n]['week_data'][i]['video']
    data_title.append(week_data_title)
    data_desc.append(week_data_desc)
    data_video.append(week_data_video)

In [468]:
data_created_by = pd.DataFrame({'title': title,
                    'created_by':created_by},
                    columns = ['title', 'created_by'])

data_detail = pd.DataFrame({     
                    'week_data_desc': data_desc,
                    'week_data_title': data_title,
                    'week_data_video': data_video
                    },
                    columns = ['week_data_desc', 'week_data_title','week_data_video'])


In [469]:
data_created_by.drop_duplicates(inplace=True)

In [470]:
len(data_created_by)

637

In [471]:
for i, row in data_created_by.iterrows():
    if not isEnglish(row.title):
        data_created_by.drop(i, inplace=True)

In [472]:
data_detail = pd.merge(data_created_by, data_detail, left_index=True, right_index=True, how="left")

In [475]:
for i, row in data_detail.iterrows():
    if len(row.week_data_video) == 0:
        data_detail.drop(i, inplace=True)

In [476]:
data_detail = data_detail.reset_index(drop=True)

In [477]:
nlp = spacy.load('en')

In [478]:
week_video_list = []
for video_data in data_detail.week_data_video:
    video_str = ''
    x = ' '.join(video_data)
    doc = nlp(x.lower())
    for token in doc:
        if not token.is_stop and token.is_alpha:
            video_str += token.lemma_
            video_str += ' '
    week_video_list.append(video_str)

In [479]:
data_detail['week_data_video_str'] = pd.Series(week_video_list)
data_detail.dropna(inplace=True)
data_detail = data_detail.reset_index(drop=True)

In [481]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorize = TfidfVectorizer(max_df=150)
bows = vectorize.fit(data_detail.week_data_video_str)
document_matrix = bows.transform(data_detail.week_data_video_str).toarray()

In [482]:
document_matrix.shape

(596, 8733)

In [484]:
f = bows.get_feature_names()

In [486]:
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(document_matrix)

In [550]:
data_detail.loc[np.argsort(result[20])[::-1], :]

,title,created_by,week_data_desc,week_data_title,week_data_video,week_data_video_str,category
20,Introduction to Cybersecurity for Business,University of Colorado System,[Understand the basics of computer security fo...,"[What is Practical Computer Security, The CIA ...","[Welcome!, About the Instructor, Glossary of C...",welcome instructor glossary computer security ...,5
13,Information Security: Context and Introduction,University of London,"[, This module introduces cryptography, which ...","[Introduction to Information Security, Introdu...","[An overview, What are you hoping to achieve i...",overview hop achieve course information securi...,5
6,Cybersecurity and Its Ten Domains,University System of Georgia,[Checkout our marquee speaker. She's the Chief...,"[Welcome and Course Overview, Introduction to ...","[Welcome to ""Cybersecurity and Its Ten Domains...",welcome cybersecurity domain welcome course im...,5
408,Cyber Security in Manufacturing,"University at Buffalo, The State University of...",[The purpose of this module is to introduce yo...,[Introduction to Digital Manufacturing Securit...,"[Course Introduction, Digital Manufacturing Se...",course introduction digital manufacture securi...,6
49,Enterprise and Infrastructure Security,New York University Tandon School of Engineering,[This module includes an introduction to many ...,"[Security Awareness, Compliance, Assessments, ...",[Introduction: What You Will Learn from This C...,introduction learn course cyber security assig...,5
588,Software Requirements Prioritization: Risk Ana...,University of Colorado System,[Once requirements have been somewhat/mostly d...,"[Risk Analysis, Assessment and Prioritization,...","[Intro to Risk Analysis, Assessment, and Prior...",intro risk analysis assessment prioritization ...,1
79,The Business of Cybersecurity Capstone,University System of Georgia,[This module provides a welcome to the course ...,[Welcome to the Management of Information Secu...,"[An overview of this course, Welcome to securi...",overview course welcome security management se...,5
33,Enterprise System Management and Security,University of Colorado System,[Welcome and Introduction to Enterprise System...,[It's not that different from my home computer...,"[Welcome!, About this course, Resources for bu...",welcome course resource build test environment...,1
1,Cryptography I,Stanford University,[Week 1. This week's topic is an overview of ...,"[Course overview and stream ciphers, Block Cip...","[Lecture slides for all six weeks, Course over...",lecture slide week course overview additional ...,5
38,Cyber Threats and Attack Vectors,University of Colorado System,[Welcome! This week we'll explore users and us...,[THREATS AND ATTACK VECTORS IN COMPUTER SECURI...,"[Introduction, About this course, About the In...",introduction course instructor digital identit...,5


In [ ]:
created_by

In [593]:
from py2neo import Graph

graph = Graph(password='pwd')

In [594]:
graph.delete_all()

2017-12-28 18:06:26,200 : INFO : > GET http://localhost:7474/db/manage/server/jmx/domain/org.neo4j
2017-12-28 18:06:26,202 : INFO : ~ Reconnecting (peer closed connection)
2017-12-28 18:06:26,204 : INFO : > GET http://localhost:7474/db/manage/server/jmx/domain/org.neo4j
2017-12-28 18:06:26,214 : INFO : < 200 OK [68955]
2017-12-28 18:06:26,216 : INFO : C: RUN 'MATCH (a) DETACH DELETE a' {}
2017-12-28 18:06:26,217 : INFO : C: PULL_ALL 
2017-12-28 18:06:26,218 : INFO : S: SUCCESS {'fields': [], 'result_available_after': 0}
2017-12-28 18:06:26,219 : INFO : S: SUCCESS {'result_consumed_after': 0, 'type': 'w'}


In [596]:
from py2neo import Node
institute_dict = {}
for created_by in set(data.created_by):
    institute_dict[created_by] = Node("Institute", name=created_by)
    graph.create(institute_dict[created_by])

2017-12-28 18:09:45,639 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'University of Michigan'}}
2017-12-28 18:09:45,640 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,647 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 1}
2017-12-28 18:09:45,648 : INFO : S: RECORD [(b'N', (1079, ['Institute'], {'name': 'University of Michigan'}))]
2017-12-28 18:09:45,650 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,651 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'MongoDB Inc.'}}
2017-12-28 18:09:45,651 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,657 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:45,657 : INFO : S: RECORD [(b'N', (1080, ['Institute'], {'name': 'MongoDB Inc.'}))]
2017-12-28 18:09:45,659 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1,

2017-12-28 18:09:45,746 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,750 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 1}
2017-12-28 18:09:45,751 : INFO : S: RECORD [(b'N', (1094, ['Institute'], {'name': 'Emory University'}))]
2017-12-28 18:09:45,752 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,752 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'Google Daydream Impact'}}
2017-12-28 18:09:45,752 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,757 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:45,757 : INFO : S: RECORD [(b'N', (1095, ['Institute'], {'name': 'Google Daydream Impact'}))]
2017-12-28 18:09:45,758 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,759 : INFO : C: RUN 'CREATE (a0:Institute

2017-12-28 18:09:45,846 : INFO : S: RECORD [(b'N', (1109, ['Institute'], {'name': 'Erasmus University Rotterdam'}))]
2017-12-28 18:09:45,847 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,848 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'Georgia Institute of Technology'}}
2017-12-28 18:09:45,848 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,852 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:45,853 : INFO : S: RECORD [(b'N', (1110, ['Institute'], {'name': 'Georgia Institute of Technology'}))]
2017-12-28 18:09:45,853 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,854 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'The Pennsylvania State University'}}
2017-12-28 18:09:45,854 : I

2017-12-28 18:09:45,946 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,946 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'The University of Chicago'}}
2017-12-28 18:09:45,946 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,953 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:45,953 : INFO : S: RECORD [(b'N', (1125, ['Institute'], {'name': 'The University of Chicago'}))]
2017-12-28 18:09:45,954 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:45,954 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'École normale supérieure'}}
2017-12-28 18:09:45,954 : INFO : C: PULL_ALL 
2017-12-28 18:09:45,959 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:45,959 : I

2017-12-28 18:09:46,042 : INFO : S: RECORD [(b'N', (1139, ['Institute'], {'name': 'The University of Melbourne, The Chinese University of Hong Kong'}))]
2017-12-28 18:09:46,043 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:46,043 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'West Virginia University'}}
2017-12-28 18:09:46,044 : INFO : C: PULL_ALL 
2017-12-28 18:09:46,048 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:46,048 : INFO : S: RECORD [(b'N', (1140, ['Institute'], {'name': 'West Virginia University'}))]
2017-12-28 18:09:46,049 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:46,049 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'University of California, San Diego, Natio

2017-12-28 18:09:46,163 : INFO : C: PULL_ALL 
2017-12-28 18:09:46,167 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:46,168 : INFO : S: RECORD [(b'N', (1154, ['Institute'], {'name': 'Northwestern University'}))]
2017-12-28 18:09:46,168 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:46,169 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'The Hong Kong University of Science and Technology'}}
2017-12-28 18:09:46,169 : INFO : C: PULL_ALL 
2017-12-28 18:09:46,174 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:46,174 : INFO : S: RECORD [(b'N', (1155, ['Institute'], {'name': 'The Hong Kong University of Science and Technology'}))]
2017-12-28 18:09:46,175 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'

2017-12-28 18:09:46,258 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'Eindhoven University of Technology'}}
2017-12-28 18:09:46,258 : INFO : C: PULL_ALL 
2017-12-28 18:09:46,264 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:46,265 : INFO : S: RECORD [(b'N', (1169, ['Institute'], {'name': 'Eindhoven University of Technology'}))]
2017-12-28 18:09:46,265 : INFO : S: SUCCESS {'result_consumed_after': 0, 'stats': {'nodes-created': 1, 'properties-set': 1, 'labels-added': 1}, 'type': 'rw'}
2017-12-28 18:09:46,266 : INFO : C: RUN 'CREATE (a0:Institute {x0})\nRETURN a0 LIMIT 1' {'x0': {'name': 'California Institute of the Arts'}}
2017-12-28 18:09:46,266 : INFO : C: PULL_ALL 
2017-12-28 18:09:46,270 : INFO : S: SUCCESS {'fields': ['a0'], 'result_available_after': 0}
2017-12-28 18:09:46,271 : INFO : S: RECORD [(b'N', (1170, ['Institute'], {'name': 'California Institute of the Arts'}))]
2017-12-28 18:09:46,271 : INFO : S: SU

In [ ]:
teach_dict = {}
for teach_by in set(data.teach_by):
    teach_dict[teach_by] = Node("Teacher", name=teach_by)
    graph.create(teach_dict[teach_by])

In [604]:
from py2neo import Relationship
for i, row in data.iterrows():
    x = teach_dict[row.teach_by]
    y = institute_dict[row.created_by]
    graph.create(Relationship(x, "from", y))
    

2017-12-28 18:31:56,207 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1321, 'y0': {}, 'x1': 1117}
2017-12-28 18:31:56,208 : INFO : C: PULL_ALL 
2017-12-28 18:31:56,928 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 706}
2017-12-28 18:31:56,928 : INFO : S: RECORD [(b'N', (1321, ['Teacher'], {'name': 'Arvind Narayanan'})), (b'N', (1117, ['Institute'], {'name': 'Princeton University'})), (b'R', (0, 1321, 1117, 'from', {}))]
2017-12-28 18:31:56,930 : INFO : S: SUCCESS {'result_consumed_after': 2, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotificatio

2017-12-28 18:31:56,985 : INFO : S: SUCCESS {'result_consumed_after': 1, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.PlannerUnavailableWarning', 'severity': 'WARNING'}], 'stats': {'relationships-created': 1}, 'type': 'rw'}
2017-12-28 18:31:56,986 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1271, 'y0': {}, 'x1': 1097}
2017-12-28 18:31:56,986 : INFO : C: PULL_ALL 
2017-12-28 18:31:56,992 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 1}
2017-12-28 18:31:56,992 : INFO : S: RECORD [(b'N', (1271, ['Teacher'], {'name': 'Jong-Moo

2017-12-28 18:31:57,035 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1526, 'y0': {}, 'x1': 1134}
2017-12-28 18:31:57,036 : INFO : C: PULL_ALL 
2017-12-28 18:31:57,041 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 1}
2017-12-28 18:31:57,042 : INFO : S: RECORD [(b'N', (1526, ['Teacher'], {'name': 'Dr Jorge Blasco Alis'})), (b'N', (1134, ['Institute'], {'name': 'University of London'})), (b'R', (31, 1526, 1134, 'from', {}))]
2017-12-28 18:31:57,043 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotifica

2017-12-28 18:31:57,087 : INFO : S: SUCCESS {'result_consumed_after': 1, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.PlannerUnavailableWarning', 'severity': 'WARNING'}], 'stats': {'relationships-created': 1}, 'type': 'rw'}
2017-12-28 18:31:57,088 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1516, 'y0': {}, 'x1': 1117}
2017-12-28 18:31:57,088 : INFO : C: PULL_ALL 
2017-12-28 18:31:57,094 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 1}
2017-12-28 18:31:57,095 : INFO : S: RECORD [(b'N', (1516, ['Teacher'], {'name': 'Dr. Nick

2017-12-28 18:31:57,131 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1480, 'y0': {}, 'x1': 1129}
2017-12-28 18:31:57,131 : INFO : C: PULL_ALL 
2017-12-28 18:31:57,132 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 1}
2017-12-28 18:31:57,133 : INFO : S: RECORD [(b'N', (1480, ['Teacher'], {'name': 'Telethia Willis'})), (b'N', (1129, ['Institute'], {'name': 'Cisco'})), (b'R', (23, 1480, 1129, 'from', {}))]
2017-12-28 18:31:57,134 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.Plann

...

2017-12-28 18:32:03,007 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.PlannerUnavailableWarning', 'severity': 'WARNING'}], 'type': 'rw'}
2017-12-28 18:32:03,007 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1233, 'y0': {}, 'x1': 1160}
2017-12-28 18:32:03,008 : INFO : C: PULL_ALL 
2017-12-28 18:32:03,009 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 0}
2017-12-28 18:32:03,009 : INFO : S: RECORD [(b'N', (1233, ['Teacher'], {'name': 'Mia Minnes'})), (b'N', (1160, ['Institute'], {'

2017-12-28 18:32:03,046 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1501, 'y0': {}, 'x1': 1112}
2017-12-28 18:32:03,046 : INFO : C: PULL_ALL 
2017-12-28 18:32:03,047 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 0}
2017-12-28 18:32:03,048 : INFO : S: RECORD [(b'N', (1501, ['Teacher'], {'name': 'Brian Winn'})), (b'N', (1112, ['Institute'], {'name': 'Michigan State University'})), (b'R', (473, 1501, 1112, 'from', {}))]
2017-12-28 18:32:03,049 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification

2017-12-28 18:32:03,083 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.PlannerUnavailableWarning', 'severity': 'WARNING'}], 'stats': {'relationships-created': 1}, 'type': 'rw'}
2017-12-28 18:32:03,084 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1376, 'y0': {}, 'x1': 1142}
2017-12-28 18:32:03,085 : INFO : C: PULL_ALL 
2017-12-28 18:32:03,087 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 0}
2017-12-28 18:32:03,087 : INFO : S: RECORD [(b'N', (1376, ['Teacher'], {'name': 'Anne Rem

2017-12-28 18:32:03,115 : INFO : C: PULL_ALL 
2017-12-28 18:32:03,116 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 0}
2017-12-28 18:32:03,117 : INFO : S: RECORD [(b'N', (1289, ['Teacher'], {'name': 'Don Patterson'})), (b'N', (1164, ['Institute'], {'name': 'University of California, Irvine'})), (b'R', (64, 1289, 1164, 'from', {}))]
2017-12-28 18:32:03,118 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Statement.PlannerUnavailableWarning', 'severity': 'WARNING'}], 'type': 'rw'}
2017-12-28 18:32:03,119 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:

2017-12-28 18:32:03,150 : INFO : C: RUN 'MATCH (a0) WHERE id(a0)={x0}\nMATCH (a1) WHERE id(a1)={x1}\nCREATE UNIQUE (a0)-[r0:from]->(a1) SET r0={y0}\nRETURN a0, a1, r0 LIMIT 1' {'x0': 1494, 'y0': {}, 'x1': 1088}
2017-12-28 18:32:03,150 : INFO : C: PULL_ALL 
2017-12-28 18:32:03,151 : INFO : S: SUCCESS {'fields': ['a0', 'a1', 'r0'], 'result_available_after': 0}
2017-12-28 18:32:03,152 : INFO : S: RECORD [(b'N', (1494, ['Teacher'], {'name': 'Andrew D. Hilton'})), (b'N', (1088, ['Institute'], {'name': 'Duke University'})), (b'R', (85, 1494, 1088, 'from', {}))]
2017-12-28 18:32:03,153 : INFO : S: SUCCESS {'result_consumed_after': 0, 'notifications': [{'title': 'The RULE planner is not available in the current CYPHER version, the query has been run by an older CYPHER version.', 'position': {'column': 1, 'offset': 58, 'line': 3}, 'description': 'CREATE UNIQUE is unsupported for current CYPHER version, the query has been executed by an older CYPHER version', 'code': 'Neo.ClientNotification.Stat

In [605]:
from vis import draw

options = {"Teacher": "name", 'Institue': 'name'}
draw(graph, options)



2017-12-28 18:32:11,057 : INFO : C: RUN '\n    MATCH (n)\n    WITH n, rand() AS random\n    ORDER BY random\n    LIMIT {limit}\n    OPTIONAL MATCH (n)-[r]->(m)\n    RETURN n AS source_node,\n           id(n) AS source_id,\n           r,\n           m AS target_node,\n           id(m) AS target_id\n    ' {'limit': 100}
2017-12-28 18:32:11,058 : INFO : C: PULL_ALL 
2017-12-28 18:32:11,083 : INFO : S: SUCCESS {'fields': ['source_node', 'source_id', 'r', 'target_node', 'target_id'], 'result_available_after': 19}
2017-12-28 18:32:11,084 : INFO : S: RECORD [(b'N', (1450, ['Teacher'], {'name': 'Min Song'})), 1450, (b'R', (423, 1450, 1097, 'from', {})), (b'N', (1097, ['Institute'], {'name': 'Yonsei University'})), 1097]
2017-12-28 18:32:11,085 : INFO : S: RECORD [(b'N', (1585, ['Teacher'], {'name': 'Christopher Brinton'})), 1585, (b'R', (57, 1585, 1117, 'from', {})), (b'N', (1117, ['Institute'], {'name': 'Princeton University'})), 1117]
2017-12-28 18:32:11,087 : INFO : S: RECORD [(b'N', (1320,

2017-12-28 18:32:11,128 : INFO : S: RECORD [(b'N', (1538, ['Teacher'], {'name': 'Noam  Nisan'})), 1538, (b'R', (84, 1538, 1098, 'from', {})), (b'N', (1098, ['Institute'], {'name': 'Hebrew University of Jerusalem'})), 1098]
2017-12-28 18:32:11,129 : INFO : S: RECORD [(b'N', (1114, ['Institute'], {'name': 'University of Copenhagen'})), 1114, None, None, None]
2017-12-28 18:32:11,129 : INFO : S: RECORD [(b'N', (1350, ['Teacher'], {'name': 'Helen Tibbo'})), 1350, (b'R', (437, 1350, 1138, 'from', {})), (b'N', (1138, ['Institute'], {'name': 'The University of North Carolina at Chapel Hill, The University of Edinburgh'})), 1138]
2017-12-28 18:32:11,130 : INFO : S: RECORD [(b'N', (1395, ['Teacher'], {'name': 'Parham Aarabi'})), 1395, (b'R', (291, 1395, 1093, 'from', {})), (b'N', (1093, ['Institute'], {'name': 'University of Toronto'})), 1093]
2017-12-28 18:32:11,132 : INFO : S: RECORD [(b'N', (1222, ['Teacher'], {'name': 'Rob A. Rutenbar'})), 1222, (b'R', (116, 1222, 1161, 'from', {})), (b'N',

2017-12-28 18:32:11,173 : INFO : S: RECORD [(b'N', (1098, ['Institute'], {'name': 'Hebrew University of Jerusalem'})), 1098, None, None, None]
2017-12-28 18:32:11,173 : INFO : S: RECORD [(b'N', (1361, ['Teacher'], {'name': 'Dr. Jerry Roth'})), 1361, (b'R', (278, 1361, 1163, 'from', {})), (b'N', (1163, ['Institute'], {'name': 'Vanderbilt University'})), 1163]
2017-12-28 18:32:11,174 : INFO : S: RECORD [(b'N', (1626, ['Teacher'], {'name': 'Marie-Laure Boucheret'})), 1626, (b'R', (200, 1626, 1115, 'from', {})), (b'N', (1115, ['Institute'], {'name': 'Institut Mines-Télécom'})), 1115]
2017-12-28 18:32:11,175 : INFO : S: RECORD [(b'N', (1433, ['Teacher'], {'name': 'Mung Chiang'})), 1433, (b'R', (56, 1433, 1117, 'from', {})), (b'N', (1117, ['Institute'], {'name': 'Princeton University'})), 1117]
2017-12-28 18:32:11,176 : INFO : S: RECORD [(b'N', (1189, ['Teacher'], {'name': 'Michael D. Ekstrand'})), 1189, (b'R', (350, 1189, 1113, 'from', {})), (b'N', (1113, ['Institute'], {'name': 'University

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(document_matrix)
inertia_list = []  
for i in range(2,15):
    kmeans = KMeans(n_clusters=i).fit(X)
    inertia_list.append(kmeans.inertia_)

import matplotlib.pyplot as plt
plt.plot(range(2,15), inertia_list, linestyle='-', marker='o')
plt.show() 

In [445]:
![image.png](attachment:image.png)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [14]:
#IsDuplicated = data_to_mysql.duplicated()

In [31]:
# duplicate_course=0
# for i in range(len(data_to_mysql)):
#     if IsDuplicated[i] == True:
#         duplicate_course=duplicate_course+1
#         #print(i)
# print(duplicate_course)

167


In [30]:
#data = data_to_mysql.drop_duplicates()

In [31]:
#data_to_mysql.head()

### --------------------------------------------------------------------------------

In [458]:
### Testing

In [464]:
# v=[]
# for n in range(len(all_data)):
#     for j in range(len(all_data[n]['week_data'])):     
#         v.append(list((all_data[n]['week_data'][j]).values())[0])
#         #a.append(pd.Series(v).str.cat(sep='')) 

In [385]:
# v= pd.Series(v).str.cat(sep='')